In [99]:
# some useful mysklearn package import statements and reloads
import importlib
import numpy as np

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation


In [ ]:
space_table = MyPyTable().load_from_file('input_data/space.csv')

space_table.remove_rows_with_missing_values()

space_dict = myutils.count_label_occurrences(space_table.get_column("is_hazardous"))
for value in sorted(space_dict.keys()):
    print(value,space_dict[value])

columns_to_remove = [0, 1, 2, 5]
space_table.column_names = [col for i, col in enumerate(space_table.column_names) if i not in columns_to_remove]
space_table.data = [
    [value for i, value in enumerate(row) if i not in columns_to_remove] 
    for row in space_table.data
]

true_list = []
false_list = []
for i, item in enumerate(space_table.data):
    if "TRUE" in item:
        true_list.append(i)
    elif "FALSE" in item:
        false_list.append(i)
np.random.shuffle(true_list)
np.random.shuffle(false_list)
full_sample = []
true_sample = [space_table.data[i] for i in true_list[:1000]]
false_sample = [space_table.data[i] for i in false_list[:1000]]
full_sample.extend(true_sample)
full_sample.extend(false_sample)
np.random.shuffle(full_sample)
space_table.data = full_sample
min_diameter_bins, min_labels = myutils.discretization(space_table.get_column(space_table.column_names.index("estimated_diameter_min")))

for instance in space_table.data:
    for i, (lower, upper) in enumerate(min_diameter_bins):
        if lower <= instance[space_table.column_names.index("estimated_diameter_min")] <= upper:
            instance[space_table.column_names.index("estimated_diameter_min")] = i 
            break

max_diameter_bins, max_labels = myutils.discretization(space_table.get_column(space_table.column_names.index("estimated_diameter_max")))

for instance in space_table.data:
    for i, (lower, upper) in enumerate(max_diameter_bins):
        if lower <= instance[space_table.column_names.index("estimated_diameter_max")] <= upper:
            instance[space_table.column_names.index("estimated_diameter_max")] = i 
            break

velocity_bins, velocity_labels = myutils.discretization(space_table.get_column(space_table.column_names.index("relative_velocity")))

for instance in space_table.data:
    for i, (lower, upper) in enumerate(velocity_bins):
        if lower <= instance[space_table.column_names.index("relative_velocity")] <= upper:
            instance[space_table.column_names.index("relative_velocity")] = i 
            break

miss_bins, miss_counts = myutils.discretization(space_table.get_column(space_table.column_names.index("miss_distance")))
for instance in space_table.data:
    for i, (lower, upper) in enumerate(miss_bins):
        if lower <= instance[space_table.column_names.index("miss_distance")] <= upper:
            instance[space_table.column_names.index("miss_distance")] = i 
            break

y = space_table.get_column(space_table.column_names.index("is_hazardous"))  
X = [row[:space_table.column_names.index("is_hazardous")] + row[space_table.column_names.index("is_hazardous") + 1:] for row in space_table.data]

tree_classifier = MyDecisionTreeClassifier()
tree_classifier.fit(X,y)

header = space_table.column_names
tree_classifier.print_decision_rules(attribute_names=header)
tree_classifier.visualize_tree(
    "space_tree.dot", "space_tree", attribute_names=header
)


FALSE 295009
TRUE 43162
['neo_id', 'name', 'absolute_magnitude', 'estimated_diameter_min', 'estimated_diameter_max', 'orbiting_body', 'relative_velocity', 'miss_distance', 'is_hazardous']
9
['estimated_diameter_min', 'estimated_diameter_max', 'relative_velocity', 'miss_distance', 'is_hazardous']
IF relative_velocity == 0 AND miss_distance == 0 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 1 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 2 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 3 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 4 THEN class = FALSE.
IF relative_velocity == 1 AND miss_distance == 0 THEN class = TRUE.
IF relative_velocity == 1 AND miss_distance == 1 THEN class = TRUE.
IF relative_velocity == 1 AND miss_distance == 2 THEN class = TRUE.
IF relative_velocity == 1 AND miss_distance == 3 THEN class = FALSE.
IF relative_velocity == 1 AND miss_distance == 4 THEN class = FALSE.
IF relative_velo